In [1]:
#! pip install mlflow boto3 awscli optuna imbalanced-learn

In [2]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://youtube-recommender-exp-bucket/5', creation_time=1763818486726, experiment_id='5', last_update_time=1763818486726, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [4]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
df = pd.read_csv(r'C:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [6]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1] as xgboost doesnt read -1
df['category'] = df.category.map({-1:2,0:0,1:1})

# step 2:Remove rows where target label asre null
df = df.dropna(subset=['category'])

ngram_range = (1,3) #trigram from previous exp
max_features = 1000 # max_feature from previous experiment

# step 3: train test split before vectorization and resampling
X_train,X_test,y_train,y_test = train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42,stratify=df['category'])

#step 4: vectorization using BOW , fit on training data only
vectorizer = CountVectorizer(ngram_range=ngram_range,max_features=max_features)
X_train_vec= vectorizer.fit_transform(X_train).astype(np.float32)
X_test_vec = vectorizer.transform(X_test).astype(np.float32)

# use undersampler method as it came out best in exp4
rus = RandomUnderSampler(random_state=42)
X_train_vec,y_train = rus.fit_resample(X_train_vec,y_train)
X_train_vec = X_train_vec.astype(np.float32)


# Ensure integer labels
#y_train = y_train.astype(int)
#y_test = y_test.astype(int)

#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparision")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)
        f1_macro = f1_score(y_test, y_pred, average="macro")
        mlflow.log_metric("f1_macro",f1_macro)
        f1_weighted = f1_score(y_test, y_pred, average="weighted")
        mlflow.log_metric("f1_weighted",f1_weighted)


        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    if metric != "support":
                        mlflow.log_metric(f"{label}_{metric}", value)

        #Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

#step 6: Optuna objective function for lightgbm
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators',50,300)
    learning_rate = trial.suggest_float('learning_rate',1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth',3,10)

    # validation split INSIDE Optuna
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train_vec, y_train, test_size=0.2, random_state=42, stratify=y_train
    )

     # convert both splits to float32
    X_tr  = X_tr.astype(np.float32)
    X_val = X_val.astype(np.float32)



    model = LGBMClassifier(
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    random_state=42,
    metric="multi_logloss",
    objective="multiclass",   
    num_class=3                  
)
    
    # Train on X_tr and evaluate on X_val
    model.fit(X_tr, y_tr)
    y_val_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_val_pred)

    return accuracy

#stage 7 :run otuna for XG boost , log in the best model only 
def run_optuna_experiment():
    study = optuna.create_study(direction= "maximize")
    study.optimize(objective_lightgbm,n_trials =30)

    #get the best parameter and log only the best parameter
    best_params = study.best_params
    best_model = LGBMClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    random_state=42,
    metric="multi_logloss",
    objective="multiclass",
    num_class=3
)

        
    # Log the best model with MLflow, passing the algo_name as "lightgbm"
    log_mlflow("LightGBM", best_model, X_train_vec, X_test_vec, y_train, y_test)

#run the experiment for lightgbm
run_optuna_experiment()



[I 2025-11-22 19:11:34,032] A new study created in memory with name: no-name-3a4e64f7-503f-4785-8252-1fd04f4ab112


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020813 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4254
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 969
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-22 19:11:37,654] Trial 0 finished with value: 0.5751452386966406 and parameters: {'n_estimators': 217, 'learning_rate': 0.0019781555758866024, 'max_depth': 5}. Best is trial 0 with value: 0.5751452386966406.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030286 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4254
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 969
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-22 19:11:38,794] Trial 1 finished with value: 0.5781763071482697 and parameters: {'n_estimators': 228, 'learning_rate': 0.003021013048424005, 'max_depth': 4}. Best is trial 1 with value: 0.5781763071482697.


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036422 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4254
[LightGBM] [Info] Number of data points in the train set: 15835, number of used features: 969
[LightGBM] [Info] Start training from score -1.098486
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Info] Start training from score -1.098675
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-11-22 19:12:11,066] Trial 29 finished with value: 0.5938368274816873 and parameters: {'n_estimators': 129, 'learning_rate': 0.004421972865052382, 'max_depth': 6}. Best is trial 21 with value: 0.7671129072998232.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015801 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total 

c:\Users\mukti\Desktop\youtube_sentiment_analysis\youtube_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
2025/11/22 19:12:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LightGBM_Undersampling_BOW_Trigram at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/5/runs/8fa9808d32394150b48ec66c45750415
🧪 View experiment at: http://ec2-51-20-65-11.eu-north-1.compute.amazonaws.com:5000/#/experiments/5
